In [1]:
from typing import Dict

from langchain.chains import (create_history_aware_retriever,
                              create_retrieval_chain)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama


class SciBot:
    def __init__(self, llm: str) -> None:
        self.store = {}

        self.llm = ChatOllama(model=llm)
        # ===============================================

        ### Contextualize question ###
        self.contextualize_q_system_prompt = (
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

        ### Answer question ###
        self.system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know. Use three sentences maximum and keep the "
            "answer concise."
            "\n\n"
            "{context}"
        )

    def ingest(self, db_path: str) -> None:
        """
        Load the database and create the conversational chain.
        """
        model = "hkunlp/instructor-xl"
        kwargs = {"device": "cpu"}
        embeddings = HuggingFaceInstructEmbeddings(
            model_name=model,
            model_kwargs=kwargs,
        )

        db = FAISS.load_local(
            folder_path=db_path,
            index_name="faiss_index",
            embeddings=embeddings,
            allow_dangerous_deserialization=True,
        )

        self.retriever = db.as_retriever(
            search_type="mmr",  # “similarity” (default), “mmr”, or “similarity_score_threshold”
            search_kwargs={"k": 6},
        )

        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        history_aware_retriever = create_history_aware_retriever(
            self.llm, self.retriever, contextualize_q_prompt
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

        self.question_answer_chain = create_stuff_documents_chain(self.llm, qa_prompt)

        self.rag_chain = create_retrieval_chain(
            history_aware_retriever, self.question_answer_chain
        )

        self.conversational_rag_chain = RunnableWithMessageHistory(
            self.rag_chain,
            self.get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )

    def get_session_history(self, session_id: str) -> ChatMessageHistory:
        """
        Get the chat history for a given session ID.
        """
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def ask(self, query: str, session_id: str = "abc123") -> Dict[str, str]:
        """
        Ask a question and get a response.
        """
        response = self.conversational_rag_chain.invoke(
            {"input": query},
            config={
                "configurable": {"session_id": session_id},
            },
        )
        return response

In [2]:
with open("../../data/llm_eval/questions.txt", "r") as f:
    questions = f.readlines()

In [3]:
llms = ["qwen2.5:3b", "qwen2.5:7b-instruct-q4_0", "llama3.1:latest", "llama3.2:latest"]
db_path = "../../data/dbs/db_instructor"

In [4]:
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
results = []
for llm in llms:
    chat = SciBot(llm=llm)
    chat.ingest(db_path)
    sample_ans = chat.ask("What is the capital of France?") # Sample question to eliminate cold start
    
    for i, question in enumerate(tqdm(questions)):
        start = time.time()
        response = chat.ask(question, session_id=str(i))
        ex_time = time.time() - start
        context = "".join(f"Document {i+1}: \n {doc.page_content} \n\n" for i, doc in enumerate(response["context"]))
        results.append([llm, question, context, response["answer"], ex_time])

/home/majkel/miniconda3/lib/python3.12/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


/home/majkel/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/majkel/miniconda3/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arb

max_seq_length  512


100%|██████████| 25/25 [04:45<00:00, 11.43s/it]


...

100%|██████████| 25/25 [13:10<00:00, 31.63s/it]


load INSTRUCTOR_Transformer


/home/majkel/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/majkel/miniconda3/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arb

max_seq_length  512


100%|██████████| 25/25 [03:16<00:00,  7.88s/it]


In [6]:
df = pd.DataFrame(results, columns=["llm", "question", "context", "answer", "ex_time"])
df

,llm,question,context,answer,ex_time
0,qwen2.5:3b,"What does the term ""learn to optimize"" mean?\n","Document 1: \n NatlSciRev ,2024,Vol.11,nwae132...","The term ""learn to optimize"" (L2O) refers to a...",9.534233
1,qwen2.5:3b,Please give some examples of metaheuristics.\n,Document 1: \n usually provide only sub-optima...,Metaheuristics include various algorithms insp...,9.287290
2,qwen2.5:3b,"What is the ""no free lunch"" theorem about?\n",Document 1: \n IEEE TRANSACTIONS ON EVOLUTIONA...,"The ""no free lunch"" (NFL) theorem states that ...",7.663871
3,qwen2.5:3b,What is the concept behind Parallel Algorithm ...,Document 1: \n training set as well as for con...,A Parallel Algorithm Portfolio (AAP) is a coll...,8.298491
4,qwen2.5:3b,Please provide some approaches to how Parallel...,"Document 1: \n algorithms, and thereby combine...","To construct a parallel algorithm portfolio, s...",11.127954
...,...,...,...,...,...
95,llama3.2:latest,What are the main benefits of AAC?\n,"Document 1: \n as expected, as N and K get lar...",Augmentative and Alternative Communication (AA...,12.816970
96,llama3.2:latest,Please provide an overview of the types of AAC...,Document 1: \n the best solver in the solver c...,There are several Augmentative and Alternative...,14.187720
97,llama3.2:latest,Are Parallel Algorithm Portfolios connected wi...,"Document 1: \n algorithms, and thereby combine...","Yes, the concept of parallel algorithm portfol...",6.678970
98,llama3.2:latest,What factors and challenges influenced the dev...,Document 1: \n training set as well as for con...,The development of the Parallel Algorithm Port...,13.886470


In [ ]:
df.to_csv("../../data/llm_eval/real_outputs.csv", index=False)